In [25]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



In [26]:
import pandas as pd

In [27]:
# PRAGMA PARAMETER

num_vehicles: int = 1
depot_index: int = 0

In [28]:
# PRAGMA INPUT

# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['locations'] = []

    df_locations = pd.read_csv(
        "data/location.csv"
    )

    for index, row in df_locations.iterrows():
        data['locations'].append([row['x_cord'] * 114, row['y_cord'] * 80])

    # read data from the config dict
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot_index

    return data


# [START distance_callback]
def create_distance_callback(data, manager):
    """Creates callback to return distance between points."""
    distances_ = {}
    index_manager_ = manager
    # precompute distance between location to have distance callback in O(1)
    for from_counter, from_node in enumerate(data['locations']):
        distances_[from_counter] = {}
        for to_counter, to_node in enumerate(data['locations']):
            if from_counter == to_counter:
                distances_[from_counter][to_counter] = 0
            else:
                distances_[from_counter][to_counter] = (
                    abs(from_node[0] - to_node[0]) +
                    abs(from_node[1] - to_node[1]))

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback
    # [END distance_callback]


# [START solution_printer]
def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print('Objective: {}'.format(assignment.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    plan_output += 'Distance of the route: {}m\n'.format(route_distance)
    print(plan_output)
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model()
    print(data)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)
    # [END routing_model]

    # Create and register a transit callback.
    # [START transit_callback]
    distance_callback = create_distance_callback(data, manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    assignment = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if assignment:
        print_solution(manager, routing, assignment)
    # [END print_solution]



In [29]:
main()

{'locations': [[114, 240], [228, 320], [342, 400], [456, 480], [570, 560], [684, 640], [798, 720], [912, 800], [1026, 880], [228, 320], [342, 400], [456, 480], [570, 560], [684, 640], [798, 720], [912, 800], [1026, 880], [228, 320], [342, 400], [456, 480], [570, 560], [684, 640]], 'num_vehicles': 1, 'depot': 0}
Objective: 3104
Route for vehicle 0:
 0 -> 17 -> 9 -> 1 -> 18 -> 10 -> 2 -> 19 -> 11 -> 3 -> 20 -> 12 -> 4 -> 21 -> 13 -> 5 -> 14 -> 6 -> 15 -> 7 -> 16 -> 8 -> 0
Distance of the route: 3104m

